# Relaties analyseren (naïeve manier)

## Data verzamelen

De relaties op de website van thuis worden getoond op basis van de voornaam. Het probleem is dat er meerdere personages met dezelfde voornaam zijn. Dus we kunnen op basis van deze informatie de relaties niet goed analyseren. Maar in een eerste stap zullen we dat toch doen, kwestie van een idee te krijgen hoe we die relatiedata kunnen verwerken

We maken gebruik van thuis\_http\_utils.get_url() om het relatiebestand te downloaden en de tekst te pakken te krijgen. Vervolgens bewaren we de relatiedata in _relaties\_namen.csv_ met behulp van thuis\_html\_utils.extract\relaties()

Deze code moeten we natuurlijk maar eenmaal uitvoeren. Voor de rest zullen we met het CSV-bestand werken.

In [1]:
from thuis_http_utils import get_url
from thuis_html_utils import extract_relaties

RELATIE_URL = "https://nergensbeterdanthuis.fandom.com/nl/wiki/Relaties"
RELATIE_BESTAND = "relaties_namen.csv"

inhoud = get_url(RELATIE_URL, True)   # inhoud = get_url(RELATIE_URL) wanneer het bestand al gedownload is
extract_relaties(RELATIE_BESTAND, inhoud)

## Een lijst met unieke namen
De namen komen meerdere keren voor. Niet alleen omdat de voornamen uniek zijn, maar ook omdat personages meerdere relaties hebben. Daarom zullen we een lijst met unieke namen maken.

In [4]:
from csv import DictReader
RELATIE_BESTAND = "relaties_namen.csv"

unieke_namen = set()
relaties = []
with open (RELATIE_BESTAND, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=";")
    for rij in reader:
        relaties.append(rij)
        unieke_namen.add(rij['persoon_1'])
        unieke_namen.add(rij['persoon_2'])

print('aantak relaties', len(relaties))
print('aantal unieke namen', len(unieke_namen))
gesorteerde_namen = list(unieke_namen)
gesorteerde_namen.sort()
print(gesorteerde_namen)

aantak relaties 467
aantal unieke namen 170
['Abel', 'Adam', 'Adil', 'Aisha', 'Alex', 'Amber', 'André', 'Angele', 'Angèle', 'Ann', 'Arne', 'Arno', 'Axel', 'Barbara', 'Bennie', 'Bert', 'Betty', 'Bianca', 'Bob', 'Bonnie', 'Bram', 'Britt', 'Carlos', 'Carolien', 'Charité', 'Chiara', 'Christine', 'Claire', 'Cois', 'Cédric', 'Daisy', 'David', 'Dieter', 'Dirk', 'Dorien', 'Dries', 'Dré', 'Eddy', 'Eline', 'Elke', 'Emma', 'Eric', 'Eva', 'Femke', 'Fien', 'Filip', 'Floris', 'Florke', 'Frank', 'Franky', 'François', 'Frederique', 'Gaby', 'Gaston', 'Geert', 'Gitta', 'Guy', 'Hanne', 'Harry', 'Herman', 'Hélène', 'Ilias', 'Ingrid', 'Isabelle', 'Ivo', 'Jacques', 'Jan', 'Jana', 'Jean-Marie', 'Jean-Pierre', 'Jef', 'Jenny', 'Jens', 'Jessica', 'Jetteke', 'Joeri', 'John', 'Joren', 'Judith', 'Julia', 'Kaat', 'Karin', 'Kasper', 'Katrien', 'Kobe', 'Kris', 'Kristoff', 'Lena', 'Leo', 'Leon', 'Leontien', 'Linda', 'Lou', 'Louise', 'Lowie', 'Luc', 'Lynn', 'Maarten', 'Madeleine', 'Manon', 'Margot', 'Marianne', 'Marie'

## relaties op basis van het volgnummer van het personage
Nu dat we een (gesorteerde) lijst met de namen van de personages hebben, kunnen we de namen in de relaties vervangen door nummers. We zullen het resultaat bewaren in een CSV-bestand

In [13]:
from csv import DictWriter
RELATIE_NR_BESTAND = "relaties_nrs.csv"
UNIEKE_NAMEN = "unieke_namen.txt"
relatie_nrs = []
for relatie in relaties:
    nr1 = gesorteerde_namen.index(relatie['persoon_1'])
    nr2 = gesorteerde_namen.index(relatie['persoon_2'])
    relatie_nrs.append({'seizoen': int(relatie['seizoen']), 'nr1': nr1, 'nr2':nr2})
with open(RELATIE_NR_BESTAND, mode='w', newline='', encoding='utf-8') as f:
    writer = DictWriter(f, delimiter=';', fieldnames=('seizoen', 'nr1', 'nr2'))
    writer.writeheader()
    writer.writerows(relatie_nrs)
with open(UNIEKE_NAMEN, mode='w', encoding='utf-8') as f:
    f.write(";".join(gesorteerde_namen))

## Een Numpy array met de relaties per seizoen
Nu alles omgezet is in nummers, kunnen we een numpy array maken met de gegevens. Het zal een driedimensionele matrix zijn. [aantal_seizoene, aantal_unieke_namen, aantal_unieke_namen]

In [20]:
import numpy as np
aantal_seizoenen = relatie_nrs[-1]['seizoen']
aantal_unieke_namen = len(gesorteerde_namen)
arr_relaties = np.zeros((aantal_seizoenen, aantal_unieke_namen, aantal_unieke_namen), dtype=np.uint8)
for relatie in relatie_nrs:
    seizoen = relatie['seizoen'] -1
    arr_relaties[seizoen, relatie['nr1'], relatie['nr2']] = 1
    arr_relaties[seizoen, relatie['nr2'], relatie['nr1']] = 1   #een relatie is wederzijds
np.info(arr_relaties)


class:  ndarray
shape:  (29, 170, 170)
strides:  (28900, 170, 1)
itemsize:  1
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x18d12296350
byteorder:  little
byteswap:  False
type: uint8


In [39]:
resultaat_arr = np.any(arr_relaties==1, axis=0)
resultaat_nr_arr = resultaat_arr.astype(np.uint8)
aantal_relaties = resultaat_nr_arr @ resultaat_nr_arr
np.argmax(aantal_relaties.diagonal())
gesorteerde_namen[9]

'Ann'

In [36]:
from thuis_html_utils import HOOFDPERSONAGE_CSV, NEVENPERSONAGE_CSV, GASTPERSONAGE_CSV,RELATIES_NAMEN_CSV
from csv import DictReader

hoofdpersonages = []
with open(HOOFDPERSONAGE_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        hoofdpersonages.append(rij)

nevenpersonages = []
with open(NEVENPERSONAGE_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        nevenpersonages.append(rij)
gastpersonages = []
with open(GASTPERSONAGE_CSV, mode='r', newline='', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        gastpersonages.append(rij)
relaties = []
with open(RELATIES_NAMEN_CSV, mode='r', encoding='utf-8') as f:
    reader = DictReader(f, delimiter=';')
    for rij in reader:
        relaties.append(rij)

print(hoofdpersonages[:5])
print(nevenpersonages[:5])
print(gastpersonages[:5])
print(relaties[:5])


[{'voornaam': 'Adil', 'achternaam': 'Bakkal', 'seizoenen': '[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]'}, {'voornaam': 'Britt', 'achternaam': 'Van Noteghem', 'seizoenen': '[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]'}, {'voornaam': 'Cédric', 'achternaam': 'Barry', 'seizoenen': '[29]'}, {'voornaam': 'Christine', 'achternaam': 'Leysen', 'seizoenen': '[24, 25, 26, 27, 28, 29]'}, {'voornaam': 'Dieter', 'achternaam': 'Van Aert', 'seizoenen': '[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]'}]
[{'voornaam': 'Abel', 'achternaam': 'Nauwelaers', 'seizoenen': '[29]'}, {'voornaam': 'Hannah', 'achternaam': 'Cremers', 'seizoenen': '[21, 22, 23, 24, 25, 26, 27, 28, 29]'}, {'voornaam': 'Jaan', 'achternaam': 'Bakkal', 'seizoenen': '[26, 27, 28, 29]'}, {'voornaam': 'Leo', 'achternaam': 'Vereken', 'seizoenen': '[25, 26, 27, 28, 29]'}, {'voornaam': 'Polly', 'achternaam': 'Bomans', 'seizoenen': '[29]'}]
[{'voornaam': 'Adam', 'achternaam': 'Kiabaté', 'seizoenen': '[22]'}, {'voor

In [37]:
personage_lijst = hoofdpersonages + nevenpersonages + gastpersonages
def bestaat_personage(voornaam, seizoen) -> bool:
    personages = [personage for personage in personage_lijst if personage['voornaam'] == voornaam]
    if len(personages) == 0:
        print(f'Personage {voornaam} niet gevonden')
        return False
    for personage in personages:
        if seizoen in personage['seizoenen']:
            return True
    print(f'Seizoen {seizoen} niet gevonden voor {voornaam}')
    return False

aantal_fout = 0
for relatie in relaties:
    if not bestaat_personage(relatie['persoon_1'], relatie['seizoen']): aantal_fout += 1
    if not bestaat_personage(relatie['persoon_2'], relatie['seizoen']): aantal_fout += 1

aantal_fout
    


0